#  Implementing Feature Engineering with Snowflake & Feature Store

In [ ]:
!pip install --quiet snowflake-connector-python pandas scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
from snowflake.connector import connect


conn = connect(
    user='SohanMeshram',
    password='****',
    account='****',
    warehouse='CHURN_PRED',
    database='CHURN_DB',
    schema='PUBLIC'
)

cursor = conn.cursor()

In [ ]:
query = "SELECT * FROM CHURN_DATA"
cursor.execute(query)

df = cursor.fetch_pandas_all()
print("Raw data extracted from Snowflake:")
print(df.head())


Raw data extracted from Snowflake:
   CUSTOMERID  GENDER  SENIORCITIZEN  PARTNER  DEPENDENTS  TENURE  \
0  7590-VHVEG  Female              0     True       False       1   
1  5575-GNVDE    Male              0    False       False      34   
2  3668-QPYBK    Male              0    False       False       2   
3  7795-CFOCW    Male              0    False       False      45   
4  9237-HQITU  Female              0    False       False       2   

   PHONESERVICE     MULTIPLELINES INTERNETSERVICE ONLINESECURITY  ...  \
0         False  No phone service             DSL             No  ...   
1          True                No             DSL            Yes  ...   
2          True                No             DSL            Yes  ...   
3         False  No phone service             DSL            Yes  ...   
4          True                No     Fiber optic             No  ...   

  DEVICEPROTECTION TECHSUPPORT STREAMINGTV STREAMINGMOVIES        CONTRACT  \
0               No          No   

In [ ]:
df.head()

,CUSTOMERID,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,...,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
0,7590-VHVEG,Female,0,True,False,1,False,No phone service,DSL,No,...,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,5575-GNVDE,Male,0,False,False,34,True,No,DSL,Yes,...,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,3668-QPYBK,Male,0,False,False,2,True,No,DSL,Yes,...,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,7795-CFOCW,Male,0,False,False,45,False,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,9237-HQITU,Female,0,False,False,2,True,No,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


In [ ]:
df.isnull().sum()

,0
CUSTOMERID,0
GENDER,0
SENIORCITIZEN,0
PARTNER,0
DEPENDENTS,0
TENURE,0
PHONESERVICE,0
MULTIPLELINES,0
INTERNETSERVICE,0
ONLINESECURITY,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CUSTOMERID        7043 non-null   object 
 1   GENDER            7043 non-null   object 
 2   SENIORCITIZEN     7043 non-null   int8   
 3   PARTNER           7043 non-null   bool   
 4   DEPENDENTS        7043 non-null   bool   
 5   TENURE            7043 non-null   int8   
 6   PHONESERVICE      7043 non-null   bool   
 7   MULTIPLELINES     7043 non-null   object 
 8   INTERNETSERVICE   7043 non-null   object 
 9   ONLINESECURITY    7043 non-null   object 
 10  ONLINEBACKUP      7043 non-null   object 
 11  DEVICEPROTECTION  7043 non-null   object 
 12  TECHSUPPORT       7043 non-null   object 
 13  STREAMINGTV       7043 non-null   object 
 14  STREAMINGMOVIES   7043 non-null   object 
 15  CONTRACT          7043 non-null   object 
 16  PAPERLESSBILLING  7043 non-null   bool   


In [ ]:
df = df.drop(['CUSTOMERID'], axis = 1)

In [ ]:
df[np.isnan(df['TOTALCHARGES'])]

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,ONLINEBACKUP,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
488,Female,0,True,True,0,False,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,True,Bank transfer (automatic),52.55,NaN,False
753,Male,0,False,True,0,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,20.25,NaN,False
936,Female,0,True,True,0,True,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,False,Mailed check,80.85,NaN,False
1082,Male,0,True,True,0,True,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,25.75,NaN,False
1340,Female,0,True,True,0,False,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,False,Credit card (automatic),56.05,NaN,False
3331,Male,0,True,True,0,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,19.85,NaN,False
3826,Male,0,True,True,0,True,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,25.35,NaN,False
4380,Female,0,True,True,0,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,20.00,NaN,False
5218,Male,0,True,True,0,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,True,Mailed check,19.70,NaN,False
6670,Female,0,True,True,0,True,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,False,Mailed check,73.35,NaN,False


In [ ]:
df.fillna(df["TOTALCHARGES"].mean())

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,ONLINEBACKUP,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
0,Female,0,True,False,1,False,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,Male,0,False,False,34,True,No,DSL,Yes,No,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,Male,0,False,False,2,True,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,Male,0,False,False,45,False,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,Female,0,False,False,2,True,No,Fiber optic,No,No,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,True,True,24,True,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,True,Mailed check,84.80,1990.50,False
7039,Female,0,True,True,72,True,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,True,Credit card (automatic),103.20,7362.90,False
7040,Female,0,True,True,11,False,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,True,Electronic check,29.60,346.45,False
7041,Male,1,True,False,4,True,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,True,Mailed check,74.40,306.60,True


In [ ]:
df[df['TENURE'] == 0].index

Index([488, 753, 936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754], dtype='int64')

In [ ]:
df.drop(labels=df[df['TENURE'] == 0].index, axis=0, inplace=True)
df[df['TENURE'] == 0].index

Index([], dtype='int64')

In [ ]:
df.isnull().sum()

,0
GENDER,0
SENIORCITIZEN,0
PARTNER,0
DEPENDENTS,0
TENURE,0
PHONESERVICE,0
MULTIPLELINES,0
INTERNETSERVICE,0
ONLINESECURITY,0
ONLINEBACKUP,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GENDER            7032 non-null   object 
 1   SENIORCITIZEN     7032 non-null   int8   
 2   PARTNER           7032 non-null   bool   
 3   DEPENDENTS        7032 non-null   bool   
 4   TENURE            7032 non-null   int8   
 5   PHONESERVICE      7032 non-null   bool   
 6   MULTIPLELINES     7032 non-null   object 
 7   INTERNETSERVICE   7032 non-null   object 
 8   ONLINESECURITY    7032 non-null   object 
 9   ONLINEBACKUP      7032 non-null   object 
 10  DEVICEPROTECTION  7032 non-null   object 
 11  TECHSUPPORT       7032 non-null   object 
 12  STREAMINGTV       7032 non-null   object 
 13  STREAMINGMOVIES   7032 non-null   object 
 14  CONTRACT          7032 non-null   object 
 15  PAPERLESSBILLING  7032 non-null   bool   
 16  PAYMENTMETHOD     7032 non-null   object 
 17  

In [ ]:
df.head()

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,ONLINEBACKUP,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
0,Female,0,True,False,1,False,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,Male,0,False,False,34,True,No,DSL,Yes,No,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,Male,0,False,False,2,True,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,Male,0,False,False,45,False,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,Female,0,False,False,2,True,No,Fiber optic,No,No,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


In [ ]:
df.columns

Index(['GENDER', 'SENIORCITIZEN', 'PARTNER', 'DEPENDENTS', 'TENURE',
       'PHONESERVICE', 'MULTIPLELINES', 'INTERNETSERVICE', 'ONLINESECURITY',
       'ONLINEBACKUP', 'DEVICEPROTECTION', 'TECHSUPPORT', 'STREAMINGTV',
       'STREAMINGMOVIES', 'CONTRACT', 'PAPERLESSBILLING', 'PAYMENTMETHOD',
       'MONTHLYCHARGES', 'TOTALCHARGES', 'CHURN'],
      dtype='object')

In [ ]:
# Encode categorical columns
cols = ['GENDER', 'PARTNER', 'DEPENDENTS', 'PHONESERVICE',
                    'MULTIPLELINES', 'INTERNETSERVICE', 'ONLINESECURITY',
                    'ONLINEBACKUP', 'DEVICEPROTECTION', 'TECHSUPPORT',
                    'STREAMINGTV', 'STREAMINGMOVIES', 'CONTRACT',
                    'PAPERLESSBILLING', 'PAYMENTMETHOD']

le = LabelEncoder()
for col in cols:
    df[col] = le.fit_transform(df[col])


In [ ]:
df.head()

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,ONLINEBACKUP,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,False
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,False
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,True
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,False
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,True


In [ ]:
df['CHURN'] = df['CHURN'].astype(int)


In [ ]:
scaler = MinMaxScaler()
df[['TENURE', 'MONTHLYCHARGES', 'TOTALCHARGES']] = scaler.fit_transform(
    df[['TENURE', 'MONTHLYCHARGES', 'TOTALCHARGES']]
)

In [ ]:
df.head()

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,ONLINEBACKUP,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
0,0,0,1,0,0.000000,0,1,0,0,2,0,0,0,0,0,1,2,0.115423,0.001275,0
1,1,0,0,0,0.464789,1,0,0,2,0,2,0,0,0,1,0,3,0.385075,0.215867,0
2,1,0,0,0,0.014085,1,0,0,2,2,0,0,0,0,0,1,3,0.354229,0.010310,1
3,1,0,0,0,0.619718,0,1,0,2,0,2,2,0,0,1,0,0,0.239303,0.210241,0
4,0,0,0,0,0.014085,1,0,1,0,0,0,0,0,0,0,1,2,0.521891,0.015330,1


In [ ]:
from snowflake.connector.pandas_tools import write_pandas


success, nchunks, nrows, _ = write_pandas(
    conn,
    df,
    table_name="FEATURE_STORE",
    auto_create_table=True,
    overwrite=True
)

print(f" Features loaded into Snowflake Feature Store table (rows: {nrows})")

 Features loaded into Snowflake Feature Store table (rows: 7032)


In [ ]:
# Retrieve features for model training
query = "SELECT * FROM FEATURE_STORE"
features_df = pd.read_sql(query, conn)


In [ ]:
features_df

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,ONLINEBACKUP,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
0,0,0,1,0,0.000000,0,1,0,0,2,0,0,0,0,0,1,2,0.115423,0.001275,0
1,1,0,0,0,0.464789,1,0,0,2,0,2,0,0,0,1,0,3,0.385075,0.215867,0
2,1,0,0,0,0.014085,1,0,0,2,2,0,0,0,0,0,1,3,0.354229,0.010310,1
3,1,0,0,0,0.619718,0,1,0,2,0,2,2,0,0,1,0,0,0.239303,0.210241,0
4,0,0,0,0,0.014085,1,0,1,0,0,0,0,0,0,0,1,2,0.521891,0.015330,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,1,0,1,1,0.323944,1,2,0,2,0,2,2,2,2,1,1,3,0.662189,0.227521,0
7028,0,0,1,1,1.000000,1,2,1,0,2,2,0,2,2,1,1,1,0.845274,0.847461,0
7029,0,0,1,1,0.140845,0,1,0,2,0,0,0,0,0,0,1,2,0.112935,0.037809,0
7030,1,1,1,0,0.042254,1,2,1,0,0,0,0,0,0,0,1,3,0.558706,0.033210,1


In [ ]:
# Train model
X = features_df.drop('CHURN', axis=1)
y = features_df['CHURN']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)

In [ ]:
rf_model = RandomForestClassifier().fit(X_train, y_train)

In [ ]:
y_pred_rf = rf_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86      2555
           1       0.65      0.51      0.57       961

    accuracy                           0.79      3516
   macro avg       0.74      0.70      0.71      3516
weighted avg       0.78      0.79      0.78      3516

